In [ ]:
from simtk import unit
import numpy as np
from tqdm import tqdm
import mdtraj as md
import nglview
from rdkit import Chem
from rdkit.Chem import AllChem
import neutromeratio
import matplotlib.pyplot as plt
import pickle
import torchani
import torch

In [ ]:
exp_results = pickle.load(open('../data/exp_results.pickle', 'rb'))

# specify the system you want to simulate
name = 'molDWRow_298'
#name = 'molDWRow_37'
#name = 'molDWRow_45'
#name = 'molDWRow_160'
#name = 'molDWRow_590'


from_mol_tautomer_idx = 1
to_mol_tautomer_idx = 2


t1_smiles = exp_results[name]['t1-smiles']
t2_smiles = exp_results[name]['t2-smiles']

# generate both rdkit mol
tautomer = neutromeratio.Tautomer(name=name, intial_state_mol=neutromeratio.generate_rdkit_mol(t1_smiles), final_state_mol=neutromeratio.generate_rdkit_mol(t2_smiles), nr_of_conformations=20)
tautomer.perform_tautomer_transformation_forward()


In [ ]:
# extract hydrogen donor idx and hydrogen idx for from_mol
model = torchani.models.ANI1ccx()

energy_function = neutromeratio.ANI1_force_and_energy(model = model,
                                          atoms = tautomer.intial_state_ligand_atoms,
                                          mol = tautomer.intial_state_ligand_atoms,
                                          use_pure_ani1ccx=True)

energy_and_force = lambda x : energy_function.calculate_force(x)


langevin = neutromeratio.LangevinDynamics(atoms = tautomer.intial_state_ligand_atoms,
                            energy_and_force= energy_and_force)



In [ ]:
n_steps = 10000
x0 = np.array(tautomer.intial_state_ligand_coords[0]) * unit.angstrom
trajectory = langevin.run_dynamics(x0, n_steps, progress_bar=True)


In [ ]:
traj_in_nm = [x.value_in_units() / unit.nanometer for x in trajectory[0]]

In [ ]:
run = 1
topology = md.load(f"../data/md_sampling/{name}/{name}_t{from_mol_tautomer_idx}.pdb").topology
ani_traj = md.Trajectory(traj_in_nm, topology)
view = nglview.show_mdtraj(ani_traj)
view